***GENERATED CODE FOR microsoftstock PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'stock'}, 'feature': 'stock', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '205', 'mean': '', 'stddev': '', 'min': 'AA', 'max': 'XOM', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'date'}, 'feature': 'date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '1/14/2011', 'max': '6/3/2011', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'high'}, 'feature': 'high', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '$101.61 ', 'max': '$99.72 ', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'low'}, 'feature': 'low', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '$10.40 ', 'max': '$99.57 ', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'close'}, 'feature': 'close', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '$10.68 ', 'max': '$99.67 ', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'next_weeks_open'}, 'feature': 'next_weeks_open', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '$10.59 ', 'max': '$99.96 ', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'next_weeks_close'}, 'feature': 'next_weeks_close', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '$10.52 ', 'max': '$99.67 ', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'open'}, 'feature': 'open', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '205', 'mean': '', 'stddev': '', 'min': '$10.89 ', 'max': '$99.16 ', 'missing': '0'}, 'transformation': 'String Indexer'})
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	microsoftstockdbfs = DBFSConnector.fetch([], {}, "5ebeb924a8835646db603100", spark, "{'url': '/Demo/Predict_Microsoft_Stock.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	microsoftstockautofe = TransformationMain.run(microsoftstockdbfs,json.dumps( {"FE": [{"transformationsData": {}, "feature": "quarter", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "205", "mean": "1.56", "stddev": "0.5", "min": "1", "max": "2", "missing": "0"}}, {"transformationsData": {"feature_label": "stock"}, "feature": "stock", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "AA", "max": "XOM", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "date"}, "feature": "date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "1/14/2011", "max": "6/3/2011", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "high"}, "feature": "high", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "$101.61 ", "max": "$99.72 ", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "low"}, "feature": "low", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "$10.40 ", "max": "$99.57 ", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "close"}, "feature": "close", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "$10.68 ", "max": "$99.67 ", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "volume", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "205", "mean": "111000384.04", "stddev": "133255769.02", "min": "10135730", "max": "889460755", "missing": "0"}}, {"transformationsData": {}, "feature": "percent_change_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "205", "mean": "0.27", "stddev": "2.47", "min": "-8.52713", "max": "6.67656", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "percent_change_volume_over_last_wk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "205", "mean": "2.86", "stddev": "31.72", "min": "-46.48313396", "max": "122.9442017", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "previous_weeks_volume", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "205", "mean": "117785015.89", "stddev": "155608988.96", "min": "11795580", "max": "1054415375", "missing": "0"}}, {"transformationsData": {"feature_label": "next_weeks_open"}, "feature": "next_weeks_open", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "$10.59 ", "max": "$99.96 ", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "next_weeks_close"}, "feature": "next_weeks_close", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "$10.52 ", "max": "$99.67 ", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "percent_change_next_weeks_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "205", "mean": "0.17", "stddev": "2.73", "min": "-15.4229", "max": "8.66347", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "days_to_next_dividend", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "205", "mean": "56.52", "stddev": "50.69", "min": "0", "max": "308", "missing": "0"}}, {"transformationsData": {}, "feature": "percent_return_next_dividend", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "205", "mean": "0.71", "stddev": "0.29", "min": "0.0677966", "max": "1.52862", "missing": "0"}, "transformation": ""}, {"transformationsData": {"feature_label": "open"}, "feature": "open", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "205", "mean": "", "stddev": "", "min": "$10.89 ", "max": "$99.16 ", "missing": "0"}, "transformation": "String Indexer"}, {"feature": "stock_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "12.08", "stddev": "8.43", "min": "0.0", "max": "29.0", "missing": "0"}}, {"feature": "date_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "9.65", "stddev": "6.74", "min": "0.0", "max": "23.0", "missing": "0"}}, {"feature": "high_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "98.08", "stddev": "59.19", "min": "0.0", "max": "200.0", "missing": "0"}}, {"feature": "low_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "95.24", "stddev": "58.93", "min": "0.0", "max": "197.0", "missing": "0"}}, {"feature": "close_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "95.19", "stddev": "59.01", "min": "0.0", "max": "197.0", "missing": "0"}}, {"feature": "next_weeks_open_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "99.04", "stddev": "59.25", "min": "0.0", "max": "201.0", "missing": "0"}}, {"feature": "next_weeks_close_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "98.08", "stddev": "59.19", "min": "0.0", "max": "200.0", "missing": "0"}}, {"feature": "open_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "205", "mean": "100.02", "stddev": "59.29", "min": "0.0", "max": "202.0", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	dataAutoML=functionRegression(microsoftstockautofe,"5ebeb924a8835646db603101", ["quarter", "stock", "date", "high", "low", "close", "volume", "percent_change_price", "percent_change_volume_over_last_wk", "previous_weeks_volume", "next_weeks_open", "next_weeks_close", "percent_change_next_weeks_price", "days_to_next_dividend", "percent_return_next_dividend"], "open")

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

